In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
class ffn(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(ffn, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [3]:
class rnn(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, num_layers=1):
        super(rnn, self).__init__()
        self.rnn_h = nn.RNN(input_dim, hidden_dim1, num_layers, nonlinearity='relu', batch_first=True)
        self.fc1 = nn.Linear(24, 18)
        self.fc2 = nn.Linear(18, 8)
        self.fc3 = nn.Linear(8, 2)

    def forward(self, x):
        out, _ = self.rnn_h(x)
        out = out.squeeze()
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [4]:
class dataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file,header=None)
        N = len(df)
        self.input = torch.tensor(df.iloc[:, :-2].values).float().reshape([N,24,1])
        self.output = torch.tensor(df.iloc[:, -2:].values).float()
    
    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]

In [5]:
def train(data_loader, model, optimizer, epoch, criterion, device):
    # set the model to training mode
    model.train()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [6]:
def test(data_loader, model, criterion, device):
    # set the model to evaluation mode
    model.eval()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        pred = model(data)
        loss = criterion(pred, target)
        
    return loss.item()

In [8]:
# Check if CUDA is available and set device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

train_set = dataset("trainset.csv")
test_set = dataset("testset.csv")

input_dim = len(train_set[0][0])
output_dim = len(test_set[0][1])
hidden_dim1 = 18
hidden_dim2 = 8
N = train_set.__len__()

# setup hyperparameters
batch_size = 512
lr = 0.01
n_epoch = 2000

train_loader = DataLoader(
                train_set,
                batch_size=batch_size
            )
test_loader = DataLoader(
                test_set,
                batch_size=batch_size
            )

model = rnn(1, 1, 12, output_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # Use Adam optimizer
criterion = nn.MSELoss()

# training and evaluation
for ep in range(n_epoch+1):
    train_loss = train(train_loader, model, optimizer, n_epoch, criterion, device)
    if ep%100 == 0:
        test_loss = test(test_loader, model, criterion, device)
        print(f"Epoch {ep}, train_loss: {train_loss}, test_loss: {test_loss}")

Epoch 0, train_loss: 3707.406005859375, test_loss: 3735.282470703125
Epoch 100, train_loss: 42.64882278442383, test_loss: 37.61775207519531
Epoch 200, train_loss: 42.38112258911133, test_loss: 37.261714935302734
Epoch 300, train_loss: 42.353912353515625, test_loss: 37.02778625488281
Epoch 400, train_loss: 42.336910247802734, test_loss: 36.93550109863281
Epoch 500, train_loss: 42.304405212402344, test_loss: 36.884132385253906
Epoch 600, train_loss: 42.273380279541016, test_loss: 36.84535598754883
Epoch 700, train_loss: 42.24209976196289, test_loss: 36.83115005493164
Epoch 800, train_loss: 42.22285842895508, test_loss: 36.81666946411133
Epoch 900, train_loss: 42.21208572387695, test_loss: 36.8077392578125
Epoch 1000, train_loss: 42.20142364501953, test_loss: 36.801788330078125
Epoch 1100, train_loss: 42.196163177490234, test_loss: 36.7968864440918
Epoch 1200, train_loss: 42.19929885864258, test_loss: 36.7883415222168
Epoch 1300, train_loss: 42.196895599365234, test_loss: 36.7776069641113

In [202]:
x,y = test_set[:]
x = x.to(device)
y = y.to(device)
print(x)

tensor([[13.1849, 13.9992, 11.7387,  ..., 16.4917, 15.4571, 14.1548],
        [10.8773,  8.6032,  6.4537,  ..., 17.7472, 16.9451, 16.2222],
        [ 6.8918,  8.4900,  5.7012,  ..., 12.9856, 12.2455, 15.3241],
        ...,
        [10.0221, 11.0008, 12.0454,  ..., 11.7404, 12.1688, 14.8853],
        [13.6667, 11.7543, 12.8841,  ..., 19.0000, 18.1854, 17.5771],
        [ 7.1838,  9.2874,  6.3407,  ..., 14.5533, 13.5534, 11.2462]],
       device='cuda:0')


In [198]:
pred = model(x)
print(f"Shape: {pred.shape}")
print(pred[0:10])

Shape: torch.Size([1000, 2])
tensor([[41.5390, 52.9439],
        [45.8918, 63.4754],
        [35.8327, 44.9116],
        [66.3797, 60.8742],
        [78.7583, 74.6940],
        [64.4542, 73.3021],
        [69.3992, 69.6439],
        [52.8433, 60.2746],
        [74.6433, 80.3555],
        [36.0487, 47.8025]], device='cuda:0', grad_fn=<SliceBackward0>)


In [199]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print(f"Final score (RMSE): {score}")

score = torch.sqrt(torch.nn.functional.mse_loss(pred, y))
print(f"Final score (RMSE) with training dataset: {score}")

Final score (RMSE): 3.489196538925171
Final score (RMSE) with training dataset: 3.489196538925171


In [200]:
# Sample predictions
for i in range(50):
    print(f"{i+1}. Actual: {y[i]}, " + f"Predicted: {pred[i]}")

1. Actual: tensor([51.0109, 61.6222], device='cuda:0'), Predicted: tensor([41.5390, 52.9439], device='cuda:0', grad_fn=<SelectBackward0>)
2. Actual: tensor([53.4241, 63.2120], device='cuda:0'), Predicted: tensor([45.8918, 63.4754], device='cuda:0', grad_fn=<SelectBackward0>)
3. Actual: tensor([36.6702, 45.9392], device='cuda:0'), Predicted: tensor([35.8327, 44.9116], device='cuda:0', grad_fn=<SelectBackward0>)
4. Actual: tensor([57.8885, 61.5989], device='cuda:0'), Predicted: tensor([66.3797, 60.8742], device='cuda:0', grad_fn=<SelectBackward0>)
5. Actual: tensor([80.0552, 75.1403], device='cuda:0'), Predicted: tensor([78.7583, 74.6940], device='cuda:0', grad_fn=<SelectBackward0>)
6. Actual: tensor([63.3119, 76.4552], device='cuda:0'), Predicted: tensor([64.4542, 73.3021], device='cuda:0', grad_fn=<SelectBackward0>)
7. Actual: tensor([75.9189, 72.5088], device='cuda:0'), Predicted: tensor([69.3992, 69.6439], device='cuda:0', grad_fn=<SelectBackward0>)
8. Actual: tensor([55.9193, 63.126

In [218]:
train_set[0]

(tensor([[15.0888],
         [12.6446],
         [11.8129],
         [11.0358],
         [ 9.2524],
         [11.6496],
         [13.5205],
         [16.3397],
         [18.6308],
         [19.0000],
         [17.2428],
         [17.1545],
         [17.8595],
         [19.0000],
         [16.7978],
         [16.4531],
         [19.0000],
         [19.0000],
         [16.8044],
         [15.9367],
         [16.7803],
         [19.0000],
         [18.7399],
         [16.9162]]),
 tensor([32.9114, 43.6894]))

In [329]:
x,y = train_set[range(4)]
x = x.reshape([4,12,2])
rnn = nn.RNN(2, 1, 1, nonlinearity='relu', batch_first=True)

In [330]:
print(x)

tensor([[[15.0888, 12.6446],
         [11.8129, 11.0358],
         [ 9.2524, 11.6496],
         [13.5205, 16.3397],
         [18.6308, 19.0000],
         [17.2428, 17.1545],
         [17.8595, 19.0000],
         [16.7978, 16.4531],
         [19.0000, 19.0000],
         [16.8044, 15.9367],
         [16.7803, 19.0000],
         [18.7399, 16.9162]],

        [[16.7910, 16.1484],
         [17.9919, 18.4010],
         [18.4195, 19.0000],
         [15.9542, 15.5293],
         [12.9345, 15.8914],
         [16.6398, 15.3505],
         [16.4115, 18.7246],
         [19.0000, 16.2859],
         [19.0000, 16.7159],
         [18.4123, 19.0000],
         [17.9352, 19.0000],
         [15.8380, 15.1843]],

        [[ 6.4991,  3.7815],
         [ 0.7069,  0.0000],
         [ 0.0000,  0.0000],
         [ 3.1526,  4.8034],
         [ 1.6431,  0.0000],
         [ 0.5529,  1.8762],
         [ 2.8325,  2.3855],
         [ 1.7471,  0.0000],
         [ 1.6782,  1.7007],
         [ 4.0980,  1.3627],
         [

In [331]:
y = rnn(x)
print(y)

(tensor([[[10.1020],
         [11.5467],
         [ 9.8191],
         [11.7825],
         [16.2867],
         [17.2246],
         [17.7225],
         [17.5446],
         [18.8048],
         [18.0711],
         [17.1387],
         [18.8598]],

        [[10.8125],
         [15.5010],
         [17.5334],
         [16.9526],
         [14.1145],
         [16.2567],
         [16.1900],
         [18.8450],
         [19.7736],
         [19.1642],
         [18.5292],
         [17.3074]],

        [[ 4.7000],
         [ 2.4055],
         [ 0.9317],
         [ 2.0303],
         [ 2.1706],
         [ 0.9205],
         [ 2.2548],
         [ 2.3441],
         [ 1.9700],
         [ 3.9325],
         [ 1.3711],
         [ 1.5058]],

        [[10.9141],
         [14.1234],
         [12.4460],
         [ 8.4726],
         [ 8.9599],
         [11.3834],
         [13.9695],
         [15.0619],
         [12.9010],
         [10.1868],
         [ 9.8489],
         [12.2044]]], grad_fn=<TransposeBackward1>), 

In [384]:
x = torch.ones([4,3,1]).squeeze()
y = torch.ones([3,2])
z = torch.matmul(x,y)
print(z.shape)

torch.Size([4, 2])
